# Caso de uso: corrección y traducción de un texto
<div class="alert alert-warning">
    
<i class="fa fa-bug"></i>
Si no queréis perder los resultados de ejemplo, **no ejecutéis** el cuaderno de jupyter sin tener una clave propia de la API de OpenAI. Estos cuadernos solo cumplen una función ilustrativa de los códigos y formas de utilizar el modelo GPT-3 con Python. En caso de disponer de una **clave**, guardadla en un archivo **.env** para mayor seguridad como **un texto entrecomillado asignado a la variable OPENAI_API_KEY**.
</div>

## Autentificación

La API de OpenAI utiliza claves de API para la autentificación. 

<div class="alert alert-danger">
    
<i class="fa fa-exclamation-circle"></i>
Recuerda que tu clave de API es un secreto. No la compartas con otros ni la expongas en ningún código del lado del cliente (navegadores, aplicaciones). Las solicitudes de producción deben dirigirse a través de su propio servidor *backend*, donde su clave de API puede cargarse de forma segura desde una variable de entorno o un servicio de gestión de claves.

</div>

Todas las solicitudes de API deben incluir su clave de API, es importante almacenar en un documento seguro la llave. Para ello, crea un archivo nuevo `.env` para almacenarla en su interior de la siguiente forma:

`OPENAI_API_KEY = "MI_API_KEY"`

Con esto, para recuperar la clave de la API, tendremos que usar el **getenv** de `os`.

In [1]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Introducción

En este cuaderno voy a tratar de utilizar a GPT-3 para dos tareas: **corregir un texto en castellano y traducirlo a inglés**. La entrada de texto estará cargada desde un documento Word y generará dos nuevos documentos: uno con la corrección y otro con la traducción.

GPT-3 solo es capaz de hacer una tarea a la vez, por lo que estas acciones requerirán de dos llamadas a la API consecutivas, donde el resultado de una será la entrada para la petición de la segunda.

Lo primero que voy a hacer, como en cualquier problema, es hacer la carga de bibliotecas que necesito para conseguir extraer el texto del documento Word, utilizaré para ello la biblioteca `docx` para Python.

Con esta biblioteca, lo que haremos es extraer los párrafos del documento Word y, en este caso, pasar cada párrafo como una petición diferenciada para GPT-3.

In [ ]:
# Instalar biblioteca si no la tenéis
#!pip3 install python-docx

In [3]:
from docx import Document

documento = Document(docx='./documentos/original.docx').paragraphs
for parrafos in documento:
    print(len(parrafos.text))
    print(parrafos.text)

88
Nunca llegué haber a aquel árbol que me dijó áquel ombre que me encontré bajo el puetne.
77
Los peros no ladravan a noche durante la fiesta a la que tube que a ver hido.


Tenemos el texto cargado en la variable *documento*, toca ahora pedirle a GPT-3 que haga la corrección de texto, estos trabajos se hacen estructurando el mensaje que le enviamos a GPT-3 a través de **Completion**. Tanto para las correcciones como traducciones, el modelo funciona y entiende correctamente la tarea a realizar si utilizamos las etiquetas **Original:** y **Corrección:**.

Además, como quiero que se ajuste al texto a traducir y no genere más texto, utilizaré como secuencia de parada el propio *salto de línea (\n)* o la palabra *Corregido:*, para que, cuando termine de recorrer el texto, interrumpa la ejecución. 

Como son tareas complejas que requieren del contexto de la entrada de texto, utilizaremos el motor *Davinci*; es recomendable ajustar el motor utilizado a la tarea que vamos a pedirle para poder mejorar el rendimiento general.

In [3]:
correcciones = []
for parrafo in documento:
    entrada = 'Original: ' + parrafo.text + '\nCorregido:'
    correccion = openai.Completion.create(engine='davinci',
                                          prompt=entrada,
                                          temperature=0.81,
                                          stop=['\n', 'Corregido:'],
                                          max_tokens = len(parrafo.text) + 5)
    correcciones.append(correccion['choices'][0]['text'])
print(correcciones)

[' Nunca llegué a aquel árbol que me dijo aquel ombre que me encontré bajo el puente.', ' Los perros no ladraban a noche durante la fiesta a la que tuve que ir.']


Aunque la corrección podría ser mejor, hay que tener en cuenta que estamos trabajando en español directamente en un modelo que se ha entrenado originalmente en inglés y que, aunque se defiende en español, no es demasiado bueno con él.
Del texto generado, podría esperarse de una corrección profesional:

```
Nunca llegué a ver el árbol del que aquel hombre que me encontré bajo el puente me habló (dijo).
Los perros no ladraban anoche durante la fiesta a la que tuve que ir.
```

Al menos, podemos utilizar el modelo para una corrección ortográfica muy básica que, en casos donde el texto requiera de mucha limpieza antes de una corrección de estilo propiamente dicha, podría ser interesante.
Por ahora, vamos a guardar la corrección que nos ha facilitado GPT-3, volviendo a utilizar la biblioteca **doc**.

In [4]:
doc_CORR = Document()
for parrafo in correcciones:
    doc_CORR.add_paragraph(parrafo)

doc_CORR.save('./documentos/correccion.docx')

Ahora que tenemos la corrección hecha y guardada, toca traducir el texto; volveremos a hacer una petición a GPT-3 como la anterior, pero esta vez diciéndole que el texto lo quiero traducir, no corregir. Además, ya que estamos, le pediré que haga la traducción al inglés de su corrección y la mía, a ver qué tal hace cada una.

Por supuesto, ambas traducciones también las guardaremos en un documento.

In [5]:
# ----------------------------------------------- TRADUCCIÓN CON GPT-3
traducciones = []
for parrafo in correcciones:
    entrada = 'Original: ' + parrafo + '\nTraducido al inglés:'
    traduccion = openai.Completion.create(engine='davinci',
                                          prompt=entrada,
                                          temperature=0.81,
                                          stop=['\n', 'Traducido al inglés:'],
                                          max_tokens = len(parrafo) + 5)
    traducciones.append(traduccion['choices'][0]['text'])
print(traducciones)

# ----------------------------------------------- GUARDADO DEL DOCUMENTO
doc_TRAD = Document()
for parrafo in traducciones:
    doc_TRAD.add_paragraph(parrafo)

doc_TRAD.save('./documentos/traduccion_de_COR-GPT3.docx')

[' I never reached that tree that the man said that I met under the bridge told me about.', ' The dogs were not barking at night in the party at which I had to go.']


Podemos observar como hay cosas que, a veces, pasan, como que añade comillas que no debería tener y similares; al ser un lenguaje estocástico, no podemos garantizar que siempre mantenga la coherencia (a veces, vamos, va a hacer lo que le dé la gana) y generará alguna respuesta que no nos interese o tengamos que revisar.

**Vamos a comprobar qué tal hace la traducción de la corrección de estilo que he hecho del texto.**

In [6]:
correccion_reyes = ['Nunca llegué a ver el árbol del que aquel hombre que me encontré bajo el puente me habló.',
                    'Los perros no ladraban anoche durante la fiesta a la que tuve que ir.']

# ----------------------------------------------- TRADUCCIÓN CON GPT-3
traducciones = []
for parrafo in correccion_reyes:
    entrada = 'Original: ' + parrafo + '\nTraducido al inglés:'
    traduccion = openai.Completion.create(engine='davinci',
                                          prompt=entrada,
                                          temperature=0.81,
                                          stop=['\n', 'Traducido al inglés:'],
                                          max_tokens = len(parrafo) + 5)
    traducciones.append(traduccion['choices'][0]['text'])
print(traducciones)

# ----------------------------------------------- GUARDADO DEL DOCUMENTO
doc_TRAD = Document()
for parrafo in traducciones:
    doc_TRAD.add_paragraph(parrafo)

doc_TRAD.save('./documentos/traduccion_de_COR-Reyes.docx')

[' I never saw the tree of that man that I met under the bridge arbore I never saw spoke.', " The dogs didn't bark last night at the party I had to attend."]


### Conclusiones

Aunque el trabajo de corrección es malo, sobre todo, si esperásemos una corrección de estilo a nivel profundo del texto; a la hora de traducir, GPT-3 realiza correctamente la traducción sin perder la coherencia del texto ni el estilo propio de cada uno de ellos (aunque este sea engorroso).

A la hora de corregir, es mejorable (al menos atendiendo al texto en español), pero podría llegar a ser útil para una primera limpieza.

También hay que tener en cuenta que, en estos casos, hemos estado trabajando con ejemplos no relacionados, cosa que no ocurre en un texto completo y de cierta complejidad. Hacer peticiones individuales de cada párrafo podría generar problemas de coherencia a lo largo de un texto de varios párrafos al perder las referencias; por lo tanto, es recomendable encontrar otra forma de hacer esta petición, cosa que veremos en el cuaderno **2.2_Ejemplo de un caso de uso_Traduccion**.

Por otro lado, existe una limitación de palabras máximas que GPT-3 puede generar, lo que significa que, aunque queramos hacer una corrección de un texto íntegro, probablemente tengamos que hacer un proceso recurrente para poder enlazar lo que va traduciendo con lo anterior.

<div class="alert alert-info">
    
<i class="fa fa-code"></i> **Este cuaderno ha sido creado con la ayuda de GPT-3.**
<hr/>
    
**Si tienes alguna duda relacionada con estos cuadernos, puedes contactar conmigo:**
Mª Reyes R.P. (Erebyel). **[Web](https://www.erebyel.es/) • [Twitter](https://twitter.com/erebyel) • [Linkedin](https://www.linkedin.com/in/erebyel/)**.
    
<hr/>
    
<i class="fa fa-plus-circle"></i> **Fuentes:**
* ***Documentación de la Beta de OpenAI***: https://beta.openai.com/docs/introduction
</div>